In [1]:
# !pip install plotly
# !pip install dash
# !pip install dash-renderer
# !pip install dash_html_components
# !pip install dach_core_components
#!pip install dash_table

In [2]:
import dash
import dash_core_components as dcc
import plotly.graph_objs as go
import dash_html_components as html
import pandas as pd
import numpy as np
from dash.dependencies import Input, Output
import dash_table

In [3]:
recovered_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
country_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_country.csv')
us_state_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/web-data/data/cases_state.csv')
us_death_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
us_confirmed_df = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')

In [4]:
# data cleaning

# renaming the df column names to lowercase
country_df.columns = map(str.lower, country_df.columns)
us_state_df.columns = map(str.lower, us_state_df.columns)
confirmed_df.columns = map(str.lower, confirmed_df.columns)
death_df.columns = map(str.lower, death_df.columns)
recovered_df.columns = map(str.lower, recovered_df.columns)
us_confirmed_df.columns = map(str.lower, us_confirmed_df.columns)
us_death_df.columns = map(str.lower, us_death_df.columns)

# changing province/state to state and country/region to country
confirmed_df = confirmed_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
recovered_df = recovered_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
death_df = death_df.rename(columns={'province/state': 'state', 'country/region': 'country'})
country_df = country_df.rename(columns={'country_region': 'country'})
us_confirmed_df = us_confirmed_df.rename(columns={'province_state': 'state', 'country_region': 'country'})
us_death_df = us_death_df.rename(columns={'province_state': 'state', 'country_region': 'country'})
us_state_df = us_state_df.rename(columns={'country_region': 'country','province_state' : 'state'})
us_state_df = us_state_df[us_state_df.country =='US']
# us_state_df.fillna(0, inplace=True)

In [5]:
death_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,33,36,36,40,42,43,47,50,57,58
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,26,26,26,27,27,27,27,28,28,30
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,375,384,392,402,407,415,419,425,432,437
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,36,37,37,37,37,40,40,40,40,41
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,2,2,2,2,2,2,2,2,2,2


In [6]:
recovered_df.head()

,state,country,lat,long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20
0,NaN,Afghanistan,33.0000,65.0000,0,0,0,0,0,0,...,131,135,150,166,179,188,188,207,220,228
1,NaN,Albania,41.1533,20.1683,0,0,0,0,0,0,...,314,327,345,356,385,394,403,410,422,431
2,NaN,Algeria,28.0339,1.6596,0,0,0,0,0,0,...,1047,1099,1152,1204,1355,1408,1479,1508,1558,1651
3,NaN,Andorra,42.5063,1.5218,0,0,0,0,0,0,...,235,248,282,309,333,344,344,344,385,398
4,NaN,Angola,-11.2027,17.8739,0,0,0,0,0,0,...,6,6,6,6,6,6,6,6,6,6


In [7]:
country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
0,Australia,2020-04-29 21:32:27,-25.0000,133.0000,6746,90,5670,986,26.496777,NaN,NaN,1.334124,36,AUS
1,Austria,2020-04-29 21:32:27,47.5162,14.5501,15402,580,12779,2043,171.011725,NaN,NaN,3.765745,40,AUT
2,Canada,2020-04-29 21:32:27,60.0010,-95.0010,52475,3137,20100,29238,138.618483,NaN,NaN,5.978085,124,CAN
3,China,2020-04-29 21:32:27,30.5928,114.3055,83940,4637,78455,848,5.975754,NaN,NaN,5.524184,156,CHN
4,Denmark,2020-04-29 21:32:27,56.0000,10.0000,9206,443,6558,2205,158.937800,NaN,NaN,4.812079,208,DNK


In [8]:
us_confirmed_df.head()

,uid,iso2,iso3,code3,fips,admin2,state,country,lat,long_,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.2710,-170.1320,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,136,136,136,136,139,141,141,141,141,141
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,14,14,14,14,14,14,14,14,14,14
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.2208,-66.5901,...,1213,1252,1298,1252,1416,1276,1307,1371,1389,1400
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.3358,-64.8963,...,53,53,53,54,54,54,55,57,57,57


In [9]:
us_death_df.head()
us_death_df.drop(columns=['population'])

,uid,iso2,iso3,code3,fips,admin2,state,country,lat,long_,...,4/19/20,4/20/20,4/21/20,4/22/20,4/23/20,4/24/20,4/25/20,4/26/20,4/27/20,4/28/20
0,16.0,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,...,0,0,0,0,0,0,0,0,0,0
1,316.0,GU,GUM,316,66.0,NaN,Guam,US,13.444300,144.793700,...,5,5,5,5,5,5,5,5,5,5
2,580.0,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.097900,145.673900,...,2,2,2,2,2,2,2,2,2,2
3,630.0,PR,PRI,630,72.0,NaN,Puerto Rico,US,18.220800,-66.590100,...,62,63,64,63,69,77,83,84,84,86
4,850.0,VI,VIR,850,78.0,NaN,Virgin Islands,US,18.335800,-64.896300,...,3,3,3,3,3,3,3,4,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3257,84070017.0,US,USA,840,NaN,Southeast Utah,Utah,US,38.996171,-110.701396,...,0,0,0,0,0,0,0,0,0,0
3258,84070018.0,US,USA,840,NaN,Southwest Utah,Utah,US,37.854472,-111.441876,...,1,1,1,1,1,1,2,2,2,2
3259,84070019.0,US,USA,840,NaN,TriCounty,Utah,US,40.124915,-109.517442,...,0,0,0,0,0,0,0,0,0,0
3260,84070020.0,US,USA,840,NaN,Weber-Morgan,Utah,US,41.271160,-111.914512,...,1,1,2,2,2,2,2,2,2,2


In [10]:
us_state_df.head(15)

,state,country,last_update,lat,long_,confirmed,deaths,recovered,active,fips,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,testing_rate,hospitalization_rate
0,Alabama,US,2020-04-29 21:32:31,32.3182,-86.9023,6847,250,NaN,6597.0,1.0,146.025814,80449.0,945.0,3.651234,84000001,USA,1715.733999,13.801665
1,Alaska,US,2020-04-29 21:32:31,61.3707,-152.4044,355,9,240.0,346.0,2.0,59.389476,19119.0,NaN,2.535211,84000002,USA,3198.499705,NaN
2,American Samoa,US,2020-04-29 21:32:31,-14.2710,-170.1320,0,0,NaN,NaN,60.0,0.000000,3.0,NaN,NaN,16,ASM,5.391708,NaN
3,Arizona,US,2020-04-29 21:32:31,33.7298,-111.4312,7209,308,1475.0,6901.0,4.0,99.042180,68813.0,1131.0,4.272437,84000004,USA,945.400130,15.688722
4,Arkansas,US,2020-04-29 21:32:31,34.9697,-92.3731,3137,57,1249.0,3080.0,5.0,121.165210,45449.0,389.0,1.817023,84000005,USA,1755.447124,12.400383
5,California,US,2020-04-29 21:32:31,36.1162,-119.6816,48111,1934,NaN,46177.0,6.0,122.705160,603139.0,NaN,4.019871,84000006,USA,1538.281633,NaN
6,Colorado,US,2020-04-29 21:32:31,39.0598,-105.3111,14316,736,2275.0,13580.0,8.0,252.628770,69449.0,2610.0,5.141101,84000008,USA,1225.538940,18.231350
7,Connecticut,US,2020-04-29 21:32:31,41.5978,-72.7554,26312,2089,NaN,24223.0,9.0,738.005103,94818.0,NaN,7.939343,84000009,USA,2659.477344,NaN
8,Delaware,US,2020-04-29 21:32:31,39.3185,-75.5071,4655,144,1173.0,4511.0,10.0,478.041907,21538.0,NaN,3.093448,84000010,USA,2211.829560,NaN
9,Diamond Princess,US,2020-04-29 21:32:31,NaN,NaN,49,0,0.0,49.0,88888.0,NaN,NaN,NaN,0.000000,84088888,USA,NaN,NaN


In [11]:
sorted_us_state_df = us_state_df.sort_values('confirmed', ascending= False)
sorted_us_state_df.head()

,state,country,last_update,lat,long_,confirmed,deaths,recovered,active,fips,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3,testing_rate,hospitalization_rate
36,New York,US,2020-04-29 21:32:31,42.1657,-74.9481,299691,23473,53238.0,276218.0,36.0,1777.418633,872481.0,65397.0,7.832401,84000036,USA,5174.543066,21.821476
34,New Jersey,US,2020-04-29 21:32:31,40.2989,-74.5210,116365,6771,15642.0,109594.0,34.0,1310.093569,241318.0,NaN,5.818760,84000034,USA,2716.875005,NaN
25,Massachusetts,US,2020-04-29 21:32:31,42.2302,-71.5301,60433,3153,NaN,57280.0,25.0,880.463395,265618.0,5758.0,5.217348,84000025,USA,3869.854651,9.527907
17,Illinois,US,2020-04-29 21:32:31,40.3495,-88.9861,50358,2213,NaN,48145.0,17.0,429.102667,256667.0,NaN,4.394535,84000017,USA,2187.070460,NaN
5,California,US,2020-04-29 21:32:31,36.1162,-119.6816,48111,1934,NaN,46177.0,6.0,122.705160,603139.0,NaN,4.019871,84000006,USA,1538.281633,NaN


In [12]:
sorted_country_df = country_df.sort_values('confirmed', ascending= False)
sorted_country_df.head()

,country,last_update,lat,long_,confirmed,deaths,recovered,active,incident_rate,people_tested,people_hospitalized,mortality_rate,uid,iso3
17,US,2020-04-29 21:32:27,40.000000,-100.00000,1036652,60475,120384,855793,314.645854,NaN,NaN,5.833684,840,USA
160,Spain,2020-04-29 21:32:27,40.463667,-3.74922,236899,24275,132929,79695,506.683990,NaN,NaN,10.246983,724,ESP
10,Italy,2020-04-29 21:32:27,41.871900,12.56740,203591,27682,71252,104657,336.726505,NaN,NaN,13.596868,380,ITA
6,France,2020-04-29 21:32:27,46.227600,2.21370,169053,24121,49117,95815,258.991733,NaN,NaN,14.268306,250,FRA
16,United Kingdom,2020-04-29 21:32:27,55.000000,-3.00000,166440,26166,853,139421,245.175721,NaN,NaN,15.720981,826,GBR


In [13]:
table_columns = sorted_country_df[['country','last_update','confirmed','recovered','deaths','active','incident_rate','mortality_rate']]
us_table_columns = sorted_us_state_df[['state','country','last_update','confirmed','recovered','deaths','active','incident_rate','mortality_rate']]

In [14]:
colors = {
    'text' : '#AF2406',
    'plot_color' : '#D3D3D3',
    'paper_color' : 'CFE9E8'
}

In [15]:
# total number of confirmed, death and recovered cases
confirmed_total = int(country_df['confirmed'].sum())
deaths_total = int(country_df['deaths'].sum())
recovered_total = int(country_df['recovered'].sum())
active_total = int(country_df['active'].sum())

In [16]:
# total number of confirmed, death and recovered cases
us_confirmed_total = int(us_state_df['confirmed'].sum())
us_deaths_total = int(us_state_df['deaths'].sum())
us_active_total = int(us_state_df['active'].sum())

In [17]:
app = dash.Dash()

In [18]:
app.layout = html.Div([
    html.H1(children = "COVID-19 CORONAVIRUS PANDEMIC",
            style = {
                'textAlign' : 'center',
                'color' : colors['text']
           }),
    html.H3(children = 'Coronavirus spread summary and tracker - World',
            style = {
                'textAlign' : 'center',
                'color' : colors['text']
           }),
    html.Img(src = "https://www.stjosephshealth.org/images/covid19-header.jpg",
            style = {
                'width' : '100%',
                'height' : '400px',
                'border': 'thin grey solid'
            }),
    html.Br(),
    html.Br(),
    html.Div([
        html.Span(children = 'Confirmed Cases: '+ str(confirmed_total),
              style = {
                  'color' : 'blue',
                  'font-size' : '30px'
           }),
        html.Span(children = 'Active Cases: '+ str(active_total),
             style = {
                  'color' : 'orange',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        html.Span(children = 'Deaths: '+ str(deaths_total),
              style = {
                  'color' : 'red',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        html.Span(children = 'Recovered: '+ str(recovered_total),
              style = {
                  'color' : 'green',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        ],
    style = {
          'textAlign' : 'center',
    }),
    html.Br(),
    html.Br(),
    html.Div([
         html.Label(children ="number of countries: ",
                   style = {
                       'color' : 'blue',
                      'font-size' : '20px'
                   }),
        dcc.Slider(
            id = 'country-slider',
            min = 1,
            max = 50,
            value = 10,
            step = 1,
            marks = {i : i for i in range(51)}
        )],
#         style ={
#             'width' : '100%'
#         }),
        ),
    html.Br(),
    html.Br(),
    dash_table.DataTable(
        id = 'table-world-cases',
        columns=[{"name": i, "id": i} for i in table_columns],
        #data=sorted_country_df.to_dict('records'),
        style_table={'overflowX': 'scroll',
                    'maxHeight': '425px',
                    'overflowY': 'scroll'},
        fixed_rows={ 'headers': True, 'data': 0 },
         style_cell={
        'height': 'auto',
        'minWidth': '120px', 'maxWidth': '140px',
        'whiteSpace': 'normal',
        'textAlign': 'left',
         'backgroundColor': 'rgb(250, 250, 250)',
        },
        style_header={
        'height' : '40px',
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
        },
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        },
        {
            'if': {'column_id': 'country'},
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'confirmed'},
            'backgroundColor': '#0AD3F7',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'deaths'},
            'backgroundColor': '#AF2406',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'recovered'},
            'backgroundColor': '#3D9970',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'active'},
            'backgroundColor': '#FC7E27',
            'fontWeight': 'bold'     
        }
        ],
    ),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-cases',
            figure = {
                'data' : [
                    go.Bar(
                        name = 'Confirmed Cases',
                        x = np.array(list(confirmed_df.iloc[:, 20:].columns)),
                        y = np.sum(np.asarray(confirmed_df.iloc[:,4:]),axis = 0),
                    ),
                    go.Bar(
                       name = 'Deaths', 
                       x = np.array(list(death_df.iloc[:, 20:].columns)),
                       y = np.sum(np.asarray(death_df.iloc[:,4:]),axis = 0),
                    ),
                    go.Bar(
                       name = 'Recoveries',
                       x = np.array(list(recovered_df.iloc[:, 20:].columns)),
                       y = np.sum(np.asarray(recovered_df.iloc[:,4:]),axis = 0),
                    )
                ],
                'layout' : go.Layout(
                    title = "Cases around the world",
                    xaxis = {'title' : 'Countries'},
                    yaxis = {'title' : 'Cases'},
                    hovermode = 'closest',
                    height=400,
                    barmode = 'stack',
                    showlegend = True
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Dropdown(
            id = 'case-type',
            options = [
                {'label' : "Confirmed Cases", 'value' : 'confirmed'},
                {'label' : "Active Cases", 'value' : 'active'},
                {'label' : "Death Cases", 'value' : 'deaths'},
                {'label' : "Recovered Cases", 'value' : 'recovered'}
            ],
            value = 'confirmed',
        )],
         style ={
            'width' : '50%'
        }),
    html.Br(),
    html.Br(),
    html.Div([ 
        dcc.Graph(
            id = 'piechart-world-cases')],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
#         'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-confirmed-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'country',
                        x=list(sorted_country_df.country),
                        y=list(sorted_country_df.confirmed),
                        mode='markers',
                        #marker = dict(color = '#0AD3F7')
                        marker = dict(
                            size =  sorted_country_df.confirmed, 
                            sizemin = 4,
                            sizemode='area',
                            sizeref = 2.*max(sorted_country_df.confirmed)/(100**2),
                            color=np.random.randn(300),
                            colorscale='Rainbow',),
                    )],
                'layout' : go.Layout(
                    title = "Confirmed cases around the world",
                    xaxis = {'title' : 'Countries',
                            'automargin' : True
                            },
                    yaxis = {'title' : 'Confirmed Cases',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-death-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'country',
                        x=list(sorted_country_df.country),
                        y=list(sorted_country_df.deaths),
                        mode='lines',
                        marker = dict(
                            color = 'red')
                    )],
                'layout' : go.Layout(
                    title = "Deaths around the world",
                    xaxis = {'title' : 'Countries',
                            'automargin' : True},
                    yaxis = {'title' : 'Deaths',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-world-recovery-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'country',
                        x=list(sorted_country_df.country),
                        y=list(sorted_country_df.recovered),
                        mode='markers+lines',
                        marker = dict(
                            color = 'green')
                    )],
                'layout' : go.Layout(
                    title = "Recoveries around the world",
                    xaxis = {'title' : 'Countries',
                            'automargin' : True},
                    yaxis = {'title' : 'Recoveries',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        html.Label(children ="Country (type a country name): ",
                   style = {
                       'color' : 'blue',
                      'font-size' : '20px'
                   }),
        dcc.Input(
            id = 'country-input',
            #placeholder = 'Input Country Name',
            type = 'text',
            value = 'World'
        )],
        style ={
            'width' : '50%'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(id = 'Check-country-cases')],
    style = {
        'border' :'solid 1px green',
    }),
    html.Br(),
    html.Br(),
    html.Div([
       dcc.Graph(
            id = 'world-geo-case-tracking',
            figure = {
                'data' : [
                    go.Choropleth(
                        locations=country_df['country'],
                        z=country_df['confirmed'].astype('int64'),
                        locationmode='country names',
                        colorscale='Reds',
                        autocolorscale=False,
                         text = country_df['country']  + '<br>' + 'Confirmed: ' + 
                        country_df['confirmed'].astype(str)  + '<br>' +'Active: ' + 
                        country_df['active'].astype(str)  + '<br>' + 'Deaths: ' + 
                        country_df['deaths'].astype(str)  + '<br>' + 'Recovered: ' + country_df['recovered'].astype(str),
                        marker_line_color='black', # line markers between states
                        colorbar_title="COVID19 confirmed cases",
                    )],
                'layout' : go.Layout(
                    title_text = "Spread of Coronavirus across the World",
                    height=500,
                    hovermode = 'closest',
                    geo = dict(
                        scope='world',
                        #projection=go.layout.geo.Projection(type = 'albers usa'),
                        showlakes=True, # lakes
                        lakecolor='rgb(255, 255, 255)'
                    ),
                )})],
    style = {
        'border' :'solid 1px green',
        #'padding' : '10px'
    }
    ),
    html.Br(),
     html.H3(children = 'Coronavirus spread summary and tracker - US',
            style = {
                'textAlign' : 'center',
                'color' : colors['text']
           }),
    html.Div([
        html.Span(children = 'US Confirmed Cases: '+ str(us_confirmed_total),
              style = {
                  'color' : 'blue',
                  'font-size' : '30px'
           }),
        html.Span(children = 'US Active Cases: '+ str(us_active_total),
             style = {
                  'color' : 'orange',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        html.Span(children = 'US Deaths: '+ str(us_deaths_total),
              style = {
                  'color' : 'red',
                  'font-size' : '30px',
                  'margin-left' : '20px'
           }),
        ],
    style = {
          'textAlign' : 'center',
    }),
    html.Br(),
    html.Br(),
    html.Div([
         html.Label(children ="number of states: ",
                   style = {
                       'color' : 'blue',
                      'font-size' : '20px'
                   }),
        dcc.Slider(
            id = 'state-slider',
            min = 1,
            max = 50,
            value = 10,
            step = 1,
            marks = {i : i for i in range(51)}
        )],
#         style ={
#             'width' : '100%'
#         }),
        ),
    html.Br(),
    html.Br(),
    dash_table.DataTable(
        id = 'table-US-cases',
        columns=[{"name": i, "id": i} for i in us_table_columns],
        #data=sorted_country_df.to_dict('records'),
        style_table={'overflowX': 'scroll',
                    'maxHeight': '425px',
                    'overflowY': 'scroll'},
        fixed_rows={ 'headers': True, 'data': 0 },
         style_cell={
        'height': 'auto',
        'minWidth': '120px', 'maxWidth': '140px',
        'whiteSpace': 'normal',
        'textAlign': 'left',
         'backgroundColor': 'rgb(250, 250, 250)',
        },
        style_header={
        'height' : '40px',
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
        },
        style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        },
        {
            'if': {'column_id': 'country'},
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'state'},
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'confirmed'},
            'color': '#0B53E6',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'deaths'},
            'color': '#AF2406',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'recovered'},
            'color': '#3D9970',
            'fontWeight': 'bold'     
        },
        {
            'if': {'column_id': 'active'},
            'color': '#FC7E27',
            'fontWeight': 'bold'     
        }
        ],
    ),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-us-cases',
            figure = {
                'data' : [
                    go.Bar(
                        name = 'Confirmed Cases',
                        x = np.array(list(us_confirmed_df.iloc[:, 21:].columns)),
                        y = np.sum(np.asarray(us_confirmed_df.iloc[:,12:]),axis = 0),
                    ),
                    go.Bar(
                       name = 'Deaths', 
                       x = np.array(list(us_death_df.iloc[:, 25:].columns)),
                       y = np.sum(np.asarray(us_death_df.iloc[:,12:]),axis = 0),
                    ),
                ],
                'layout' : go.Layout(
                    title = "Cases around the US",
                    xaxis = {'title' : 'States'},
                    yaxis = {'title' : 'Cases'},
                    hovermode = 'closest',
                    height=400,
                    barmode = 'stack',
                    showlegend = True
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Dropdown(
            id = 'us-case-type',
            options = [
                {'label' : "Confirmed Cases", 'value' : 'confirmed'},
                {'label' : "Active Cases", 'value' : 'active'},
                {'label' : "Death Cases", 'value' : 'deaths'},
            ],
            value = 'confirmed',
        )],
         style ={
            'width' : '50%'
        }),
    html.Br(),
    html.Br(),
    html.Div([ 
        dcc.Graph(
            id = 'piechart-US-cases')],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
#         'padding' : '30px'
        }),
        html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-us-confirmed-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'state',
                        x=list(sorted_us_state_df.state),
                        y=list(sorted_us_state_df.confirmed),
                        mode='markers',
                        marker = dict(
                            size =  sorted_country_df.confirmed, 
                            sizemin = 4,
                            sizemode='area',
                            sizeref = 2.*max(sorted_country_df.confirmed)/(100**2),
                            color=np.random.randn(300),
                            symbol = 'square',
                            colorscale='Rainbow',),
                    )],
                'layout' : go.Layout(
                    title = "Confirmed cases around US",
                    xaxis = {'title' : 'States',
                            'automargin' : True
                            },
                    yaxis = {'title' : 'Confirmed Cases',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        }),
    html.Br(),
    html.Br(),
    html.Div([
        dcc.Graph(
            id = 'Check-us-death-cases',
            figure = {
                'data' : [
                    go.Scatter(
                        name = 'state',
                        x=list(sorted_us_state_df.state),
                        y=list(sorted_us_state_df.deaths),
                        mode='markers+lines',
                        marker = dict(
                            color = 'red')
                    )],
                'layout' : go.Layout(
                    title = "Deaths around US",
                    xaxis = {'title' : 'States',
                            'automargin' : True},
                    yaxis = {'title' : 'Deaths',
                            'automargin' : True},
                    height=600,
                    hovermode = 'closest'
                )})],
        style ={
        'backgorund-color' : colors['plot_color'],
        'border' :'solid 1px green',
        #'padding' : '30px'
        }),
    html.Br(),
    html.Br(),
    html.Div([
       dcc.Graph(
            id = 'usa-geo-case-tracking',
            figure = {
                'data' : [
                    go.Scattergeo(
                        lat = us_state_df['lat'],
                        lon = us_state_df['long_'],
                        text = us_state_df['state']  + '<br>' + 'Confirmed: ' + 
                        us_state_df['confirmed'].astype(str)  + '<br>' +'Active: ' + 
                        us_state_df['active'].astype(str)  + '<br>' + 'Deaths: ' + 
                        us_state_df['deaths'].astype(str)  + '<br>' + 'Recovered: ' + us_state_df['recovered'].astype(str),
                        mode = 'markers',
                        marker = dict(
                            size =  us_state_df.confirmed/5, 
                            sizemin = 4,
                            sizemode='area',
                            sizeref = 2.*max(us_state_df.confirmed)/(100**2),
                            color = 'rgba(135, 206, 250, 0.8)',
                            line = dict(
                            color = 'rgb(250,0,0)',
                              width = 2
                            )
                       ),
                        hoverinfo = 'text',
                        name = 'state'
                    )],
                'layout' : go.Layout(
                    title_text = "Spread of Coronavirus across the US",
                    height=500,
                    hovermode = 'closest',
                    geo = dict(
                        scope = 'usa',
                        showcoastlines=True, coastlinecolor="RebeccaPurple",
                        showland=True, landcolor="LightGrey",
                        projection = dict(type = 'albers usa'),
                    )
                )})],
    style = {
        'border' :'solid 1px green',
        #'padding' : '10px'
    }
    ),
],
style = {
    #'background-color' : '#BBCDF2'
    'padding' : '20px'
})
    


In [19]:
@app.callback(
    Output('table-world-cases', 'data'),
    [Input('country-slider', 'value')])
def display_country_cases_table(number):
    return sorted_country_df.head(number).to_dict('records')


In [20]:
@app.callback(
    Output('table-US-cases', 'data'),
    [Input('state-slider', 'value')])
def display_country_cases_table(number):
    return sorted_us_state_df.head(number).to_dict('records')

In [21]:
@app.callback(
    Output('piechart-world-cases', 'figure'),
    [Input('case-type', 'value')])
def plot_world_info(caseType):
    if(caseType == 'confirmed'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.confirmed,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    elif(caseType == 'active'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.active,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    elif(caseType == 'deaths'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.deaths,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    else:
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_country_df.country,
            values = sorted_country_df.recovered,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    return {
        'data': [trace],
        'layout': dict(
            title = str(caseType) + ' cases around the world',
            showlegend = True,
            hovermode = 'closest',
            uniformtext_minsize=8,
            uniformtext_mode='hide',
        )
    }
    

In [22]:
@app.callback(
    Output('piechart-US-cases', 'figure'),
    [Input('us-case-type', 'value')])
def plot_world_info(caseType):
    if(caseType == 'confirmed'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_us_state_df.state,
            values = sorted_us_state_df.confirmed,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    elif(caseType == 'active'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_us_state_df.state,
            values = sorted_us_state_df.active,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    elif(caseType == 'deaths'):
        trace = go.Pie(
            name = str(caseType) + ' cases',
            labels = sorted_us_state_df.state,
            values = sorted_us_state_df.deaths,
            hole = 0.6,
            hoverinfo = 'label+value+percent',
            textposition = 'inside'
        )
    return {
        'data': [trace],
        'layout': dict(
            title = str(caseType) + ' cases around US',
            showlegend = True,
            hovermode = 'closest',
            uniformtext_minsize=8,
            uniformtext_mode='hide',
        )
    }
    

In [23]:
@app.callback(
    Output('Check-country-cases', 'figure'),
    [Input('country-input', 'value')])
def plot_country_info(country):
    labels = ['confirmed','deaths','recovered']
    colors = ['orange','red','green']
    mode_size = [7, 7, 7]
    line_size = [4, 4, 4]
    
    df_list = [confirmed_df, death_df, recovered_df]
    
    fig = go.Figure();
    
    for i, df in enumerate(df_list):
        if country.lower() == 'world':
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df.iloc[:,4:]),axis = 0)
            
        else:    
            x_data = np.array(list(df.iloc[:, 20:].columns))
            y_data = np.sum(np.asarray(df[df['country'].str.lower() == country.lower()].iloc[:,20:]),axis = 0)
            
        fig.add_trace(go.Scatter(x=x_data, y=y_data, mode='lines+markers',
        name=labels[i],
        line=dict(color=colors[i], width=line_size[i]),
        connectgaps=True,
        text = "Total " + str(labels[i]) +": "+ str(y_data[-1])
        ));
    
    fig.update_layout(
        title="COVID 19 cases of " + country,
        xaxis_title='Date',
        yaxis_title='No. of Cases',
        margin=dict(l=20, r=20, t=40, b=20),
        height = 500,        
    );
    return fig

In [ ]:
if __name__ == '__main__':
    app.run_server(port = 4048,debug=False,use_reloader=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:4048/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Apr/2020 18:19:24] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:19:24] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:19:24] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:19:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:19:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:19:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:19:24] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:19:25] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:24:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:24:15] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:24:15] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/2020 18:24:15] "GET /_favicon.ico?v=1.11.0 HTTP/1.1" 200 -
127.0.0.1 - - [29/Apr/20